<a href="https://colab.research.google.com/github/Loop3D/2022_iamg_workshop/blob/main/2a_m2l_wa_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2a m2l wa example
### Installing dependencies
This notebook will use two main libraries from the Loop project
1. LoopStructural -    
https://github.com/Loop3D/LoopStructural   
https://gmd.copernicus.org/articles/14/3915/2021/   
2. map2loop -    
https://github.com/Loop3D/map2loop-2   
https://gmd.copernicus.org/articles/14/5063/2021/

The following code blocks will install the required dependencies for the Loop libraries into this colab environment. 




## map2loop + LoopStructural

In this first example we will use buid a model from geological data provided by a Geoserver instance of Geological Survey of Western Asutralia (GSWA) data from the Hamersley Province in  Western Australia and the online Digital Terrain Model and Stratigraphic Database of Australia from Geoscience Australia (GA). 

Five datasets are used:

1. The 1:500K GSWA Interpreted Basement Geology Map
2. The 1:500K GSWA Interpreted Basement Linear Structural features Map
3. The GSWA WAROX structural obervations database
4. The SRTM-based GA Digital Terrain Map of Australia
5. Extracts from the GA Australian Stratigraphic Units Database

In [ ]:
!pip install rasterio
!pip install git+https://github.com/geopandas/geopandas.git@v0.10.2
!pip install hjson
!pip install owslib
!pip install git+https://github.com/Loop3D/map2model_cpp.git
!pip install git+https://github.com/Loop3D/LoopProjectFile.git
!pip install pygeos
!pip install mplstereonet
!pip install lavavu-osmesa
!pip install git+https://github.com/Loop3d/LoopStructural
!pip install git+https://github.com/Loop3d/map2loop-2.git
!pip install beartype

In [ ]:
bbox_3d = {'minx': 553150,
     'miny': 7455000,
     'maxx': 584300,
     'maxy': 7483000,
     'base': -4800,
     'top': 1200}

In [ ]:
import os
import hjson
from map2loop.m2l_enums import VerboseLevel
from map2loop.project import Project

from datetime import datetime
nowtime=datetime.now().isoformat(timespec='minutes')   
out_dir=nowtime.replace(":","-").replace("T","-")


proj = Project( 
                 loopdata_state = "WA",
            project_path=out_dir,
    project_crs='EPSG:28350',
    verbose_level=VerboseLevel.NONE,
    working_projection='EPSG:28350',
                )

proj.update_config(
                    out_dir=out_dir,
                    bbox_3d=bbox_3d,
                    project_crs='EPSG:28350',
                    # overwrite='true',
                    # run_flags={'fault_dip':-999},
                    # quiet='None',
    
#                     loopFilename='test.loop3d'
                  )


In [ ]:
proj.run()

In [ ]:
minx,miny,maxx,maxy = proj.config.bbox
model_base = proj.config.bbox_3d['base']
model_top = proj.config.bbox_3d['top']

In [ ]:
f=open(proj.config.tmp_path+'/bbox.csv','w')
f.write('minx,miny,maxx,maxy,lower,upper\n')
ostr='{},{},{},{},{},{}\n'.format(minx,miny,maxx,maxy,model_base,model_top)
f.write(ostr)
f.close()

In [ ]:
from LoopStructural import GeologicalModel
fault_params = {'interpolatortype':'FDI',
                'nelements':1e4,
#                'step':10,
#                'fault_buffer':0.2,
#                 'force_mesh_geometry':True,
#                 'solver':'pyamg',
#                 overprints:overprints,
#                 'cpw':1,
#                 'gpw':5,
               }
foliation_params = {'interpolatortype':'FDI' , # 'interpolatortype':'PLI',
                    'nelements':5e4,  # how many tetras/voxels
                    'buffer':2.,  # how much to extend nterpolation around box
#                     'solver':'pyamg',
#                     'damp':True
                    # 'npw':0,
                    # 'regularisation':0.5
                   }


model, m2l_data = GeologicalModel.from_map2loop_directory(proj.config.project_path,
                                                              evaluate=False,
                                                          fault_params=fault_params,
                                                          rescale=False,
#                                                           vectorscale=1,
                                                          foliation_params=foliation_params)

In [ ]:
model.update()

In [ ]:
from LoopStructural.visualisation import LavaVuModelViewer

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

view = LavaVuModelViewer(model)
view.add_model_surfaces()
view.interactive()

In [ ]:
view.display()